## Testing the sequential segmentation algorithm on real data

In [1]:
%matplotlib inline

import sys
sys.path.append('/home/paulo/github')

import time
import datetime

import numpy as np
import pandas as pd

from OceanPod import OceanPod 
from SeqSeg.SeqSeg import SeqSeg

wavfolder = '/home/paulo/github/bayeseg/Data/'
savefolder = '/home/paulo/github/bayeseg/Output/'
filelist = ['2015.01.30_02.02.56.wav', '2015.02.02_07.50.49.wav', '2015.02.08_11.26.39.wav']


# Creates object to read wave files and segments
op = OceanPod(wavfolder)

## Stabilization of MCMC 

In [ ]:
mcsteps = [5000, 10000, 15000, 20000, 50000]

alpha = 0.1
beta = 0.0005

# Pick a file
file = filelist[0]

nruns = 30

ss = SeqSeg()
fs, wave = op.readFile(file)
ss.feed_data(wave)

results = []
for mciter in mcsteps:
    ss.initialize(beta, alpha, mciter, mciter, nchains = 1)
    resiter = []
    for i in range(nruns):
        t, tdur = ss.segments(minlen = 5000, res = 11025, normalize = False)
        resiter.append([t, tdur])
        print("MC steps = " + str(mciter) + ", Nseg = " + str(len(t)+1) + ", tdur = " + str(tdur))
    
    results.append([mciter, np.mean([r[0] for r in resiter]), np.std([r[0] for r in resiter]), np.mean([r[1] for r in resiter]), np.std([r[1] for r in resiter])])

## Applying the segmentation to the real signal files

In [ ]:
# MCMC parameters

mciter = 20000
mcburn = 20000
nchains = 1


In [ ]:
# Beta grid

betamin = 0.000001
betamax = 0.00001
betamin = 1.175e-05
betamax = 1.175e-05
nstep = 0
bdelta = (betamax - betamin) / max(nstep, 1)
betalist = [betamin + d*bdelta for d in range(nstep+1)]


In [ ]:
# Alpha grid

alphamin = 0.1
alphamax = 0.1

nstep = 0
adelta = (alphamax - alphamin) / max(nstep, 1)
alphalist = [alphamin + d*adelta for d in range(nstep+1)]

In [ ]:
# Main loop
ss = SeqSeg()
normalize = True
nruns = 1

tresults = []
results = []
cont = 1
total = len(filelist) * len(alphalist) * len(betalist)
for file in filelist:
    fs, wave = op.readFile(file)
    ss.feed_data(wave)
    for alpha in alphalist:
        for beta in betalist:
            ss.initialize(beta, alpha, mciter, mcburn, nchains)
            resiter = []
            for i in range(nruns):
                t, tdur = ss.segments(minlen = 11025, res = 11025, normalize = normalize, verbose = False)
                nseg = len(t) + 1
                resiter.append([nseg, tdur])
                tresults.append([file, t])
            meannseg = np.mean([r[0] for r in resiter])
            stdseg = np.std([r[0] for r in resiter])
            tmean = np.mean([r[1] for r in resiter])
            print("({:.4%})".format(cont/total) + file + ", a = {:.2}".format(alpha) + ", b = {:.6}".format(beta) + ", Nseg = " + str(meannseg) + ", Nsegstd = " + str(stdseg) + ", t = {:.2}".format(tmean))
            cont = cont + 1
            results.append([file, alpha, beta, meannseg, stdseg, tmean])
            
            
    